In [1]:
!pip install tensorflow opencv-python firebase-admin numpy

                                              0.0/120.3 kB ? eta -:--:--
     ---                                      10.2/120.3 kB ? eta -:--:--
     ---------                             30.7/120.3 kB 435.7 kB/s eta 0:00:01
     ---------                             30.7/120.3 kB 435.7 kB/s eta 0:00:01
     ------------                          41.0/120.3 kB 196.9 kB/s eta 0:00:01
     -------------------------             81.9/120.3 kB 353.1 kB/s eta 0:00:01
     ---------------------------------    112.6/120.3 kB 435.7 kB/s eta 0:00:01
     ------------------------------------ 120.3/120.3 kB 413.8 kB/s eta 0:00:00
                                              0.0/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
             

In [9]:
import cv2
from firebase_admin import credentials, firestore, initialize_app

# Initialize Firebase Admin
cred_path = 'C:/Users/hp/Desktop/Crater Co/Attendance App/attendance-facerecog-firebase-adminsdk-pqhqh-a047efc55c.json'
cred = credentials.Certificate(cred_path)
#initialize_app(cred)
db = firestore.client()

# Prompt for Employee Name
employee_name = input("Enter employee's name: ")

# Capture Image from Webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret:
    # Process frame if necessary
    img_name = "employee_image.jpg"
    cv2.imwrite(img_name, frame)  # Save image
    
    # Add to Firebase
    doc_ref = db.collection('employees').document(employee_name)
    doc_ref.set({
        'name': employee_name,
        'image_path': img_name
    })
else:
    print("Failed to capture image")

cap.release()


Enter employee's name: lala


In [11]:
import cv2
import firebase_admin
from firebase_admin import credentials, firestore, storage, initialize_app

# Initialize Firebase Admin
cred_path = 'C:/Users/hp/Desktop/Crater Co/Attendance App/attendance-facerecog-firebase-adminsdk-pqhqh-a047efc55c.json'

# Check if app already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    default_app = initialize_app(cred, {
        'storageBucket': 'gs://attendance-facerecog.appspot.com'
    })
else:
    default_app = firebase_admin.get_app()

db = firestore.client()
bucket = storage.bucket()

# Function to upload image to Firebase Storage
def upload_image(image_path):
    blob = bucket.blob(image_path)
    blob.upload_from_filename(image_path)
    # Make the blob publicly accessible
    blob.make_public()
    return blob.public_url

# Prompt for Employee Name
employee_name = input("Enter Name: ")

# Capture Image from Webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret:
    # Process frame if necessary
    img_name = f"{employee_name}.jpg"
    cv2.imwrite(img_name, frame)  # Save image locally
    
    # Upload to Firebase Storage
    image_url = upload_image(img_name)
    
    # Add to Firestore
    doc_ref = db.collection('employees').document(employee_name)
    doc_ref.set({
        'name': employee_name,
        'image_url': image_url  # Store the URL of the uploaded image
    })
    print(f"Uploaded {employee_name}'s info successfully.")
else:
    print("Failed to capture image")

cap.release()


ValueError: Storage bucket name not specified. Specify the bucket name via the "storageBucket" option when initializing the App, or specify the bucket name explicitly when calling the storage.bucket() function.